In [1]:
import matplotlib.pyplot as plt
import nltk
import re
import regex
import string
import numpy as np
import pandas as pd
import unidecode
import seaborn as sns
import contractions
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.stem.porter import *
from nltk.tokenize import word_tokenize
from sklearn import metrics
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from wordcloud import WordCloud
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

In [2]:
df = pd.read_csv("preprocessed-metacritics-total.csv")

In [3]:
df.head()

,Movie name,Review,Created at,Score,Genre
0,Arrival,"['denis', 'villeneuve', 'shows', 'us', 'all', ...","OCT 3, 2022",1.0,Mistery
1,Arrival,"['amy', 'adams', 'gives', 'a', 'superb', 'perf...","MAR 7, 2022",1.0,Mistery
2,Arrival,"['this', 'movie', 'is', 'not', 'for', 'everyon...","DEC 6, 2019",1.0,Mistery
3,Arrival,"['arrival', 'is', 'one', 'of', 'my', 'favorite...","APR 3, 2020",1.0,Mistery
4,Arrival,"['i', 'do', 'not', 'think', 'this', 'movie', '...","MAR 2, 2020",1.0,Mistery


In [11]:
df.head()

,Movie name,Review,Created at,Score,Genre
0,Arrival,"['denis', 'villeneuve', 'shows', 'us', 'all', ...","OCT 3, 2022",1.0,Mistery
1,Arrival,"['amy', 'adams', 'gives', 'a', 'superb', 'perf...","MAR 7, 2022",1.0,Mistery
2,Arrival,"['this', 'movie', 'is', 'not', 'for', 'everyon...","DEC 6, 2019",1.0,Mistery
3,Arrival,"['arrival', 'is', 'one', 'of', 'my', 'favorite...","APR 3, 2020",1.0,Mistery
4,Arrival,"['i', 'do', 'not', 'think', 'this', 'movie', '...","MAR 2, 2020",1.0,Mistery


In [10]:
df.drop('stemmed', axis=1, inplace=True)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(df["Review"],
                                                    df["Score"],
                                                    test_size = 0.3,
                                                    random_state = 42,
                                                    stratify=df["Score"])
print('Train data: ', X_train.shape, y_train.shape)
print('Test data: ', X_test.shape, y_test.shape)

Train data:  (4536,) (4536,)
Test data:  (1944,) (1944,)


In [8]:
# Bag of Words
count_vect = CountVectorizer()  # Convercao das strings para uma matriz de contagem dos tokens

# Tranformacao das string em uma matriz de string-termo (strings e termos das strings)
# Extracao da contagem de tokens das strings usando o vocabulario anterior
X_train_bow = count_vect.fit_transform(X_train)
X_test_bow = count_vect.transform(X_test)

print("X_train_bow shape: ",X_train_bow.shape)
print("X_test_bow shape: ",X_test_bow.shape,"\n")


inst_scaler = preprocessing.StandardScaler(with_mean=False)
print("Média do dataset de treino antes do standardization: ",X_train_bow.mean())
X_train_bow = inst_scaler.fit_transform(X_train_bow)
print("Média após:",X_train_bow.mean(),"\n")

inst_scaler = preprocessing.StandardScaler(with_mean=False)
print("Média do dataset de teste antes do standardization: ",X_test_bow.mean())
X_test_bow = inst_scaler.fit_transform(X_test_bow)
print("Média após:",X_test_bow.mean())

X_train_bow shape:  (4536, 20559)
X_test_bow shape:  (1944, 20559) 

Média do dataset de treino antes do standardization:  0.0049041224580728855
Média após: 0.03206966833266205 

Média do dataset de teste antes do standardization:  0.004652673816219386
Média após: 0.02416135389039293


In [15]:
# Load a dataset (for example, the df_analise dataset)

from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC


# SVM Classifier
svm_clf = SVC()
svm_clf.fit(X_train_bow, y_train)
svm_predictions = svm_clf.predict(X_test_bow)
print("SVM Accuracy:", classification_report(y_test, svm_predictions))

# Naive Bayes Classifier
nb_clf = GaussianNB()
nb_clf.fit(X_train_bow.toarray(), y_train)
nb_predictions = nb_clf.predict(X_test_bow.toarray())
print("Naive Bayes Accuracy:", classification_report(y_test, nb_predictions))

# Random Forest Classifier
rf_clf = RandomForestClassifier()
rf_clf.fit(X_train_bow, y_train)
rf_predictions = rf_clf.predict(X_test_bow)
print("Random Forest Accuracy:", classification_report(y_test, rf_predictions))

# KNN Classifier
knn_clf = KNeighborsClassifier()
knn_clf.fit(X_train_bow, y_train)
knn_predictions = knn_clf.predict(X_test_bow)
print("KNN Accuracy:", classification_report(y_test, knn_predictions))

SVM Accuracy:               precision    recall  f1-score   support

        -1.0       0.83      0.27      0.41       648
         0.0       0.48      0.22      0.30       648
         1.0       0.40      0.88      0.55       648

    accuracy                           0.46      1944
   macro avg       0.57      0.46      0.42      1944
weighted avg       0.57      0.46      0.42      1944

Naive Bayes Accuracy:               precision    recall  f1-score   support

        -1.0       0.48      0.49      0.48       648
         0.0       0.39      0.32      0.35       648
         1.0       0.47      0.54      0.50       648

    accuracy                           0.45      1944
   macro avg       0.45      0.45      0.45      1944
weighted avg       0.45      0.45      0.45      1944

Random Forest Accuracy:               precision    recall  f1-score   support

        -1.0       0.66      0.60      0.63       648
         0.0       0.49      0.53      0.51       648
         1.0   